In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
topcv = pd.DataFrame(columns = ['Title','Company', 'Address', 'Description', 'Year_exp', 'Sex', 'Field', 'Skill', 'Company_or_Brand'])
for i in range(1, 81): 
    print(i)
    link_page = 'https://www.topcv.vn/tim-viec-lam-it-phan-mem-c10026?salary=0&exp=0&company_field=0&page=' + str(i)
    req = requests.get(link_page)
    page_obj = BeautifulSoup(req.text, 'html.parser')
    
    list_url_company = []
    list_url_brand = []
    for element in page_obj.find_all('a', {'target': '_blank'}, href=True):
        if 'viec-lam/' in element['href']:
            list_url_company.append(element['href'])
        if '/brand/' in element['href'] and '/tuyen-dung/' in element['href']:
            list_url_brand.append(element['href'])
    list_url_company = list(set(list_url_company))
    list_url_brand = list(set(list_url_brand))

    for url in list_url_company:
        try:
            req = requests.get(url)
            page_obj = BeautifulSoup(req.text, 'html.parser')
            title, company, address = [element for element in page_obj.find('div', {'class': 'col-sm-9'}).text.split('\n') if element != ''][:3]
            description = page_obj.find('div', {'id': 'col-job-left'}).text
            year_exp = page_obj.find("strong", string="Yêu cầu kinh nghiệm: ").find_next('span').text
            sex = page_obj.find("strong", string="Yêu cầu giới tính: ").find_next('span').text
            field = page_obj.find("h4", string="Ngành nghề").parent.text
            try:
                skill = page_obj.find("h4", string="Kỹ năng").parent.text
            except:
                skill = ''

            topcv.loc[len(topcv)] = [title, company, address, description, year_exp, sex, field, skill, 'Company']
        
        except:
            print(url)
    
    for url in list_url_brand:
        try:
            req = requests.get(url)
            page_obj = BeautifulSoup(req.text, 'html.parser')
            title = page_obj.find('div', {'class': 'col-sm-10'}).text.replace('\n', '')
            company = page_obj.find('div', {'id': 'company-name'}).find('h1').text
            address = page_obj.find('span', {'title': 'Địa chỉ làm việc'}).text
            description = page_obj.find('div', {'class': 'job-data'}).text
            year_exp = page_obj.find('span', {'title': 'Yêu cầu kinh nghiệm'}).text
            sex = page_obj.find('span', {'title': 'Giới tính'}).text
            field = page_obj.find("h4", string="Ngành nghề").parent.text
            try:
                skill = page_obj.find("h4", string="Kỹ năng").parent.text
            except:
                skill = ''
        except:
            print(url)

        topcv.loc[len(topcv)] = [title, company, address, description, year_exp, sex, field, skill, 'Brand']

1
2
3
4
5
6
7
8
9
10
https://www.topcv.vn/viec-lam/nhan-vien-ke-toan-tong-hop/395451.html?ta_source=JobSearchList
11
https://www.topcv.vn/viec-lam/lap-trinh-vien-software-developer/145771.html?ta_source=JobSearchList
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
https://www.topcv.vn/brand/sapo/tuyen-dung/nhan-vien-thiet-ke-ui-ux-ui-ux-designer-j431872.html?ta_source=JobSearchList
34
35
36
37
38
https://www.topcv.vn/viec-lam/junior-sales-representative-tieng-anh-thanh-thao-duoc-dao-tao-100/430623.html?ta_source=JobSearchList
39
40
41
42
43
44
45
46
47
48
49
50
https://www.topcv.vn/viec-lam/lap-trinh-asp-net-chap-nhan-khong-co-kinh-nghiem/427913.html?ta_source=JobSearchList
51
52
53
54
55
56
57
https://www.topcv.vn/viec-lam/lap-trinh-vien-developer-php-tai-quan-1/424653.html?ta_source=JobSearchList
58
59
60
61
62
63
64
https://www.topcv.vn/viec-lam/thuc-tap-sinh-php-developer-co-luong-ho-tro-thuc-tap/423916.html?ta_source=JobSearchList
65
https://www.topcv.vn/viec-lam

In [3]:
topcv.tail()

,Title,Company,Address,Description,Year_exp,Sex,Field,Skill,Company_or_Brand
2144,"Lập Trình C, C++",Công ty Cổ phần viễn thông FPT,"Lô L29B-31B-33B đường Tân Thuận, KCX Tân Thuậ...",\nMÔ TẢ CÔNG VIỆC\nPhát triển phần mềm ứng dụn...,1 năm,Không yêu cầu,\nNgành nghề\n\nIT Phần cứng / Mạng\n\n\nIT ph...,\nKỹ năng\n\nC#\n\n\nC++\n\n,Brand
2145,Kỹ Thuật Viên SEO Cao Cấp,Công ty TNHH Truyền Thông & Dữ liệu Sáng Tạo Việt,"số 1 Lương Yên, Hai Bà Trưng, Hà Nội",\nMÔ TẢ CÔNG VIỆC\n-\tThực hiện tối ưu website...,1 năm,Không yêu cầu,\nNgành nghề\n\nBáo chí / Truyền hình\n\n\nIT ...,,Brand
2146,Quản Lý Kinh Doanh Tỉnh [thái Nguyên/ Thanh H...,Công ty cổ phần Công nghệ Sapo,"số 1 Lương Yên, Hai Bà Trưng, Hà Nội",\nMÔ TẢ CÔNG VIỆC\n-\tThực hiện tối ưu website...,1 năm,Không yêu cầu,\nNgành nghề\n\nBáo chí / Truyền hình\n\n\nIT ...,,Brand
2147,Nhân Viên Chăm Sóc Khách Hàng,Công ty TNHH Buy2sell Việt Nam,"239 Phố Huế, Hà Nội/ 1026 Nguyễn Văn Linh, Ph...",\nMÔ TẢ CÔNG VIỆC\n1. Chụp hình sản phẩm trưng...,Dưới 1 năm,Không yêu cầu,\nNgành nghề\n\nKinh doanh / Bán hàng\n\n\nDịc...,,Brand
2148,Chuyên Viên Kinh Doanh B2B,Công ty TNHH Buy2sell Việt Nam,"239 Phố Huế, Hà Nội/ 1026 Nguyễn Văn Linh, Ph...",\nMÔ TẢ CÔNG VIỆC\n-\tPhát triển thông tin sản...,1 năm,Không yêu cầu,\nNgành nghề\n\nKinh doanh / Bán hàng\n\n\nTư ...,,Brand


In [4]:
topcv.to_csv('topcv_ITphanmem-24-5.csv', index=False)

## Merge data files

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
cd /content/drive/MyDrive/Data Analyst/TopCV/Data

/content/drive/MyDrive/Data Analyst/TopCV/Data


In [7]:
import os
import pandas as pd 
files = [file for file in os.listdir() if 'csv' in file]

# Concatenate many single files into one
data = pd.read_csv('./' + files[0])
for file in files:
    other_data = pd.read_csv('./' + file)
    data = pd.concat([data, other_data], ignore_index=True)

In [9]:
data.to_csv('./topcv.csv', index=False)